# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод--по-предобработке" data-toc-modified-id="Вывод--по-предобработке-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод  по предобработке</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-RandomForestClassifier" data-toc-modified-id="Модель-RandomForestClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель RandomForestClassifier</a></span></li><li><span><a href="#Модель-CatBoostClassifier" data-toc-modified-id="Модель-CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель CatBoostClassifier</a></span></li><li><span><a href="#Вывод-по-обучению-моделей" data-toc-modified-id="Вывод-по-обучению-моделей-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Вывод по обучению моделей</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка

In [1]:
import time
import pandas as pd
import numpy as np
import re
from pymystem3 import Mystem 

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from tqdm.notebook import tqdm
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

Открытие файла и знакомство с данными

In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('toxic_comments.csv')

In [4]:
display(df.head())
df.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Проверка на наличие явных дубликатов строк

In [5]:
df.duplicated().sum()

0

Проверка количества позитивных и негативных комментариев

In [6]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [7]:
class_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
class_ratio

8.841344371679229

Классы несбалансированы. Отношение 1:8.84. Проведем поиск лучшего способа балансировки и сравним качество.

1. Изменение весов в модели обучения
2. Ресемплирование с уменьшением класса 0

Ресемплирование с увеличением класса 1 не будем использовать из-за громоздкого набора данных.

Подготовим признаки и целевой признак перед обучением.

In [8]:
def clear_text(text):
    text = text.lower()  
    clear_text  = re.sub(r"(?:\n|\r)", " ", text)
    clear_text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    return clear_text

In [9]:
df['clear_text'] = df['text'].apply(clear_text)
display(df.head(15))
df = df.drop(['text'], axis=1)

,Unnamed: 0,text,toxic,clear_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanationwhy the edits made under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,daww he matches this background colour im seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war its j...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,morei cant make any real suggestions on improv...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...
5,5,"""\n\nCongratulations from me as well, use the ...",0,congratulations from me as well use the tools ...
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...
8,8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense was offensive to yo...
9,9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


In [10]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    x = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return " ".join(x)

In [11]:
%%time
tqdm.pandas()
df['text_lemma'] = df['clear_text'].apply(lemmatize_text)

CPU times: user 54.4 s, sys: 166 ms, total: 54.5 s
Wall time: 1min 49s


Деление датафрейма на две выборки: обучающую (75%) и тестовую (25%)

In [12]:
X_train_clear, X_test_clear = train_test_split(df['text_lemma'], test_size=.25, random_state=12345)
y_train, y_test = train_test_split(df['toxic'], test_size=.25, random_state=12345)

X_train_clear.shape, X_test_clear.shape, y_train.shape, y_test.shape

((119469,), (39823,), (119469,), (39823,))

Очистка текста от стоп-слов

In [13]:
%time
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

X_train = count_tf_idf.fit_transform(X_train_clear)
X_test = count_tf_idf.transform(X_test_clear)
X_train.shape, X_test.shape

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


((119469, 212462), (39823, 212462))

### Вывод  по предобработке 

- Данные импортированы с удалением лишнего индекса при чтении
- Явных дубликатов не обнаружено
- Очисили текст от стоп-слов
- В данных наблюдается большой дисбаланс классов
- Данные поделены на две выборки: обучающую и тестовую

## Обучение

Выделение констант для обучения моделей

In [14]:
N_SPLITS = 3
RS = [12345]

Создание вызываемого объекта для оценки

In [15]:
score = make_scorer(f1_score)

### Модель LogisticRegression

In [16]:
%%time

params_lr = {"class_weight": ['balanced'], 
             "random_state": RS,
             "max_iter": [100, 150, 200],
             "C": [0.5, 1.0, 1.5]
            }
model_lr = LogisticRegression()
model_lr_rs = RandomizedSearchCV(
    model_lr, cv=N_SPLITS, scoring=score, 
    param_distributions=params_lr, n_iter=N_SPLITS, verbose=True
)
model_lr_rs.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
CPU times: user 4min 8s, sys: 4min 48s, total: 8min 56s
Wall time: 12min 46s


RandomizedSearchCV(cv=3, estimator=LogisticRegression(), n_iter=3,
                   param_distributions={'C': [0.5, 1.0, 1.5],
                                        'class_weight': ['balanced'],
                                        'max_iter': [100, 150, 200],
                                        'random_state': [12345]},
                   scoring=make_scorer(f1_score), verbose=True)

In [17]:
display('Лучший результат f1-меры:', model_lr_rs.best_score_)
model_lr_rs.best_params_

'Лучший результат f1-меры:'

0.74180342183828

{'random_state': 12345, 'max_iter': 200, 'class_weight': 'balanced', 'C': 1.5}

### Модель RandomForestClassifier

In [18]:
%%time

params = {"max_depth": [5, 10, 15], 
          "n_estimators": [30, 90, 150], 
          "random_state": RS, 
          "class_weight": ['balanced']
         }
model_rfc = RandomForestClassifier()

model_rfc_rs = RandomizedSearchCV(model_rfc, cv=N_SPLITS, scoring=score, param_distributions=params)
model_rfc_rs.fit(X_train, y_train)

CPU times: user 10min 41s, sys: 0 ns, total: 10min 41s
Wall time: 28min 30s


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(),
                   param_distributions={'class_weight': ['balanced'],
                                        'max_depth': [5, 10, 15],
                                        'n_estimators': [30, 90, 150],
                                        'random_state': [12345]},
                   scoring=make_scorer(f1_score))

In [19]:
display('Лучший результат f1-меры:', model_rfc_rs.best_score_)
model_rfc_rs.best_params_

'Лучший результат f1-меры:'

0.372045802629347

{'random_state': 12345,
 'n_estimators': 150,
 'max_depth': 15,
 'class_weight': 'balanced'}

### Модель CatBoostClassifier

In [20]:
%%time

classificator = CatBoostClassifier(verbose=False, iterations=200)
classificator.fit(X_train, y_train)
cv_f1_CBC = cross_val_score(classificator,
                                         X_train, 
                                         y_train, 
                                         cv=N_SPLITS, 
                                         scoring='f1').mean()
print('F1 на cv', cv_f1_CBC)

F1 на cv 0.7216382118465088
CPU times: user 28min 31s, sys: 19.3 s, total: 28min 50s
Wall time: 41min 48s


### Вывод по обучению моделей

- При обучении Логистической Регрессии получено значение f1=0.742 за 12min 46s обучения
- При обучении Случайного Леса получено значение f1=0.372 за 28min 30s обучения
- При обучении CatBoost получено значение f1=0.722 за 41min 48s обучения

С заданием справились одна модель *LogisticRegression*

## Тестирование

In [21]:
predict = model_lr_rs.predict(X_test)
f1_score(y_test, predict)

0.7441911203228972

## Выводы

- На начальном этапе мы ознакомились с данными. Посмотрели есть ли явные дубликаты, посмотрели на сами данные.
- Далее мы занялись обработкой данных. Сначала мы очистили данные с текстом от ненужного - переносы строк, ненужные пробелы, знаки.
- Затем мы лемматизировали данные.
- Выявив дисбаланс в данных мы апсэмплировали выборку для лучшей обучаемости модели.
- Так как перед нами стояла задача классификации, то мы выбрали логистическую регрессию. Но не стали сразу обучать её на данных;
- Сначала мы векторизировали данные, потом поделили выборки на обучающую и тестовую.
- Далее мы приступили к подбору гиперпараметров, подобрав которые мы смогли перейти к главному шагу этой работы - обучение модели.
- Обучив модель и подставив гиперпараметры мы получили значение 0.75.